# create initial conditions for  simplified models

author: [Mathieu Renzo](mrenzo@flatironinstitute.org)

Assumes the models are computed with `approx21_plus_cr56.net` which
has the following isotopes

     1   neut
     2   h1
     3   prot
     4   he3
     5   he4
     6   c12
     7   n14
     8   o16
     9   ne20
    10   mg24
    11   si28
    12   s32
    13   ar36
    14   ca40
    15   ti44
    16   cr48
    17   cr56
    18   fe52
    19   fe54
    20   fe56
    21   co56
    22   ni56

In [ ]:
import sys
from pathlib import Path
import socket
if socket.gethostname() == "ccalin010.flatironinstitute.org":
    plotFunc_root = "/mnt/home/mrenzo/codes/python_stuff/plotFunc/"
elif socket.gethostname() == "t490s":
    plotFunc_root = "/home/math/Documents/Research/codes/plotFunc/"
else:
    print("you need plotFunc from https://github.com/mathren/plotFunc")
sys.path.append(plotFunc_root)
from MESAreader import *
from utilsLib import getFinalProfileLOGS, getTerminationCode
from plotDefaults import *
import matplotlib.pyplot as plt
from lib_plot_bin import plot_HRD, plot_rho_mass, get_radius,\
    plot_mdot_t, plot_omega_mass, \
    get_omega_div_omega_crit, plot_moment_of_inertia, get_Z_from_hfile,\
    get_folder_from_file, get_resolution_from_file, plot_n_modnum, \
    plot_J_m, plot_Jinside_m, plot_J_tot, plot_J_Tc, get_mass_rot_res, \
    plot_surface_rotation_time, plot_omega_div_omega_crit, plot_BE_r, \
    get_three_colormaps, get_age_from_profile, get_BE_from_pfile, plot_BE_r, \
    annotate_radii_hrd, get_ax_from_pfile
from lib_simplified_profiles import get_M_boundary, mk_simplified_profile_from_pfile, get_xq, mk_line
from tqdm import tqdm    
%load_ext autoreload
%autoreload 2

In [ ]:
set_plot_defaults_from_matplotlibrc(plotFunc_root)

In [ ]:
init_model = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/simplified_profiles/20_rot0_to_TAMS/LOGS/TAMS.data"

for offset in [1e-4,1e-3,1e-2,1e-1, 0]:
    delta_M_boundary, max_M_boundary, min_M_boundary = get_M_boundary(init_model)
    print("offset",offset,  delta_M_boundary, max_M_boundary, min_M_boundary)

# Fixed inner boundary

In [ ]:
dm = -1
m_out = max_M_boundary+dm
mk_simplified_profile_from_pfile(init_model, outfile=None, plot=True, m_in=min_M_boundary, m_out=m_out)

# Setup not fixing `min_M_boundary`

In [ ]:
delta_M_boundary, max_M_boundary, min_M_boundary = get_M_boundary(init_model)
print("input", delta_M_boundary, max_M_boundary, min_M_boundary)
print("====================================================")

i = 0
for dm in np.linspace(0,4,11):
    if dm == 0: continue
    m_in = min_M_boundary+dm
    m_out = max_M_boundary-dm
    if m_in <= m_out:
        mk_simplified_profile_from_pfile(init_model, outfile=None, plot=True, m_in=m_in, m_out=m_out)
        i+=1
    else:
        pass
    #     break
print(i)

In [ ]:
def get_output_name_from_pfile(pfile, string="20", output_root=""):
    """ give a MESA profile determines the filename for the relax files necessary to create fake models
    Parameters:
    ----------
    pfile: `string` profile*.data
    string: `optional` to prepend to the file name
    output_root: `string` path to prepend to the filename
    Returns:
    -------
    outfile: `string` path (if providing output_root) or filename of the MESA input files
    """ 
    # print(colored(pfile.split('/')[-1],"blue"))
    if "10Rsun.data" == pfile.split('/')[-1]:
        outfile = output_root+"/"+string+"_10Rsun_input"
    elif "100Rsun.data" == pfile.split('/')[-1]:
        outfile = output_root+"/"+string+"_100Rsun_input"
    elif "200Rsun.data" == pfile.split('/')[-1]:
        outfile = output_root+"/"+string+"_200Rsun_input"
    elif "300Rsun.data" == pfile.split('/')[-1]:
        outfile = output_root+"/"+string+"_300Rsun_input"
    elif "500Rsun.data" == pfile.split('/')[-1]:
        outfile = output_root+"/"+string+"_500Rsun_input"
    elif "1000Rsun.data" == pfile.split('/')[-1]:
        outfile = output_root+"/"+string+"_1000Rsun_input"
    else:
        raise ValueError("can't guess radius")
    return outfile

# outfile = get_output_name_from_pfile(pfile)

In [ ]:
# for pfile in glob.glob("/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/binaries/Z_0.0019/m1_20.0000_m2_17.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/LOGS2/*Rsun.data"):
#     # print(pfile)
#     output_root  = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/binaries/Z_0.0019/m1_20.0000_m2_17.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1/"
#     output_fname = get_output_name_from_pfile(pfile, output_root=output_root)
    # print(output_fname.split('/')[-1])
    # mk_simplified_profile_from_pfile(pfile, outfile=output_fname)    

In [ ]:
# test profile
fig=plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])

def plot_model(pfile, ax, **plot_kwargs):
    delta_M_boundary,max_M_boundary, min_M_boundary = get_M_boundary(pfile)
    src, col = getSrcCol(pfile)
    
    r = np.log10(src[:, col.index("radius")]*Rsun_cm)
    mass = src[:, col.index("mass")]
    s = src[:, col.index("entropy")]
    ind = (mass <max_M_boundary) & (mass >min_M_boundary)
    ax.axvspan(min(r[ind]),max(r[ind]), alpha=0.5, fc="#808080", **plot_kwargs)
    ax.plot(r, s, **plot_kwargs)


plot_model(pfile, ax, lw=3)
plot_model("/tmp/simpliified_profile/LOGS/profile2.data",ax)
# plot_BE_r(pfile, ax, alpha_th=1, lw=3, c="k")
# plot_BE_r("/tmp/simplified_profile/LOGS/profile2.data", ax, alpha_th=1, lw=3, c="r", ls="--")
plot_log_ratio_BE_r(pfile, "/tmp/simplified_profile/LOGS/profile2.data", ax)


# ----------------------------------------------------
# Grid of models from TAMS of single 20Msun non-rotating

In [ ]:
init_model = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/simplified_profiles/grid/20_rot0_to_TAMS/LOGS/TAMS.data"

fig=plt.figure(figsize=(20,10))
gs = gridspec.GridSpec(100, 110)
ax1 = fig.add_subplot(gs[:,:50])
ax2 = fig.add_subplot(gs[:,60:])
# # ax3 = fig.add_subplot(gs[:,80:110])

src, col = getSrcCol(init_model)
s = src[:, col.index("entropy")]
h1 = src[:, col.index("h1")]
he4 = src[:, col.index("he4")]
m = src[:, col.index("mass")]
ax1.plot(m,s)
ax2.plot(m, h1)
ax2.plot(m, he4, ls='--')

ax1.set_xlabel(r"$\mathrm{Mass\ [M_\odot]}$")
ax2.set_xlabel(r"$\mathrm{Mass\ [M_\odot]}$")
ax2.set_ylabel(r"$X_i$")
ax1.set_ylabel(r"entropy")

# original model transition region
delta_M_boundary,max_M_boundary, min_M_boundary = get_M_boundary(init_model)
ax1.axvspan(min_M_boundary, max_M_boundary, zorder=0, alpha=0.5)
ax2.axvspan(min_M_boundary, max_M_boundary, zorder=0, alpha=0.5)

for i in np.linspace(-10,10, 11):
    dm = 0.1*i #Msun
    M_boundary = (max_M_boundary+dm)-(min_M_boundary-dm)
    outfile = "/mnt/home/mrenzo/Templates/binary_mesa/simplified_profile/entropy_chem_profiles/"+f"{M_boundary:.2f}"+"_TAMS_profile_input"
    mk_simplfied_profile_from_pfile(init_model, outfile=outfile, plot=False, m_in=min_M_boundary-dm, m_out=max_M_boundary+dm)

# make unperturbed one
mk_simplfied_profile_from_pfile(init_model, outfile=outfile, plot=False, m_in=min_M_boundary-dm, m_out=max_M_boundary+dm)
